In [1]:
# %%writefile senate_vote_scraper.py
from pymongo import MongoClient
import pprint 
import pandas as pd 
import copy
from bs4 import BeautifulSoup
import pprint
import requests
import datetime

In [58]:
# https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_115_1.htm
# https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_101_1.htm
# can I employ multithreading to get these quicker?

root_url = 'https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu'


In [ ]:
cong_ids = range(101, 116)
years = range(1989, 2019)

In [14]:
empty_row = {
    'congress_id': None,
    'session': None,
    'vote_id': None, 
    'issue': None, 
    'result': None, 
    'question': None, 
    'desc': None, 
    'date': None,
    'year': None, 
    'vote_results': None
}

In [5]:
site_url = '{}_{}_1.htm'.format(root_url, cong_ids[0])
req = requests.get(site_url)
tstamp = datetime.datetime.now().strftime('%m-%d-%Y %H:%M:%S')
stat_code = req.status_code
if stat_code != 200:
    print('_______________')
    print('_______________')
    print('Error requesting {}'.format(site_url))
    print('Request Status Code: {}, {}'.format(stat_code, tstamp))
    sleep(3)

if stat_code == 200:            
#     final_roll = get_final_roll_id(root_url, yr)
#     get_table_summary(root_url, yr, final_roll)
    
    # use BeautifulSoup to find the data we need.
    soup = BeautifulSoup(req.content, 'lxml')
    table = soup.find('table')
    rows = table.find_all('tr')
    

In [87]:
yr_cong_dict.values().index('1989')

AttributeError: 'dict_values' object has no attribute 'index'

In [ ]:
for row in rows[1:]:
        new_row = copy.copy(empty_row)
        columns = row.find_all('td')
        new_row['congress_id'] = cong_ids[0]
        new_row['session'] = None
        new_row['vote_id'] = re.sub(r'[^\x00-\x7F]+', ' ' ,columns[0].text).strip().split(' ')[0]
        new_row['issue'] = columns[3].text.replace('.', ' ').replace('  ', ' ').upper().strip()
        new_row['result'] = columns[1].text
        new_row['question'] = columns[2].text.split(':', 1)[0]
        new_row['desc'] = columns[2].text.split(':', 1)[1][1:]
        new_row['date'] = re.sub(r'[^\x00-\x7F]+', ' ' ,columns[4].text).strip()
        new_row['year'] = yr_cong_dict[]
        new_row['vote_results'] = None
        
        
        
        
        
    

In [19]:
columns = rows[1].find_all('td')
for col in columns:
    print('--------')
    print(col.text.strip())

--------
312 (99-0)
--------
Agreed to
--------
On the Resolution: S. Res. 217; A resolution deploring the escalating violence in El Salvador; condemning the killing of innocent civilians, including six Jesuit priests; and calling for an immediate cessation of hostilities.
--------
S.Res. 217
--------
Nov 21


In [ ]:
import re

In [57]:
columns[2].text.split(':', 1)[1][1:]

'S. Res. 217; A resolution deploring the escalating violence in El Salvador; condemning the killing of innocent civilians, including six Jesuit priests; and calling for an immediate cessation of hostilities.'

In [48]:
re.sub(r'[^\x00-\x7F]+', ' ' ,columns[0].text).strip().split(' ')[0]

'312'

In [12]:
columns = rows[1].find_all('a')
for col in columns:
    print('--------')
    print(col)

--------
<a href="/legislative/LIS/roll_call_lists/roll_call_vote_cfm.cfm?congress=101&amp;session=1&amp;vote=00312">312 (99-0)</a>
--------
<a href="http://www.congress.gov/bill/101st-congress/senate-resolution/217">S.Res. 217</a>
